In [1]:
import pandas as pd
import numpy as np

In [9]:
dados = pd.read_csv("../data/files/dados_reduzidos.csv", sep=",", encoding="utf-8")
dados_generos = pd.read_csv("../data/files/data_by_genres.csv", sep=",", encoding="utf-8")
dados_anos = pd.read_csv("../data/files/data_by_year.csv", sep=",", encoding="utf-8")

In [24]:
dados_generos.head(2)

,mode,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.3616,-31.514333,0.040567,75.336500,0.103783,27.833333,6
1,1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.1310,-16.854000,0.076817,120.285667,0.221750,52.500000,5


In [23]:
dados_anos.head(2)

,mode,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,1921,0.886896,0.418597,260537.166667,0.231815,0.344878,0.20571,-17.048667,0.073662,101.531493,0.379327,0.653333,2
1,1,1922,0.938592,0.482042,165469.746479,0.237815,0.434195,0.24072,-19.275282,0.116655,100.884521,0.535549,0.140845,10


In [22]:
dados.head(2)

,valence,year,acousticness,artists,danceability,duration_ms,energy,id,instrumentalness,liveness,loudness,name,popularity,release_date,speechiness,tempo,artists_song
0,0.0594,1921,0.982,Sergei Rachmaninoff,0.279,831667,0.211,4BJqT0PrAfrxzMOxytFOIz,0.878,0.665,-20.096,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954,Sergei Rachmaninoff - Piano Concerto No. 3 in ...
1,0.0394,1921,0.961,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,0.328,500062,0.166,1o6I8BglA6ylDMrIELygv1,0.913,0.101,-14.850,Gati Bali,5,1921,0.0339,110.339,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...


In [32]:
dados = dados[dados.year >= 2000]
dados["year"].unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020],
      dtype=int64)

In [14]:
dados.shape

(115992, 20)

In [16]:
dados = dados.drop(["explicit", "key", "mode"], axis=1)

In [17]:
dados.shape

(115992, 17)

In [18]:
dados.isnull().sum()

valence              0
year                 0
acousticness         0
artists             12
danceability         0
duration_ms          0
energy               0
id                   0
instrumentalness     0
liveness             0
loudness             0
name                 0
popularity           0
release_date         0
speechiness          0
tempo                0
artists_song         0
dtype: int64

In [19]:
dados.isna().sum()

valence              0
year                 0
acousticness         0
artists             12
danceability         0
duration_ms          0
energy               0
id                   0
instrumentalness     0
liveness             0
loudness             0
name                 0
popularity           0
release_date         0
speechiness          0
tempo                0
artists_song         0
dtype: int64

In [21]:
dados_generos.head(2)

,mode,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.3616,-31.514333,0.040567,75.336500,0.103783,27.833333,6
1,1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.1310,-16.854000,0.076817,120.285667,0.221750,52.500000,5


In [26]:
dados_generos = dados_generos.drop(["key", "mode"], axis=1)
dados_generos.head(2)

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.3616,-31.514333,0.040567,75.336500,0.103783,27.833333
1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.1310,-16.854000,0.076817,120.285667,0.221750,52.500000


In [27]:
dados_generos.isna().sum()

genres              0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
popularity          0
dtype: int64

In [28]:
dados_generos.isnull().sum()

genres              0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
popularity          0
dtype: int64

In [29]:
dados_anos.head()

,mode,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,1921,0.886896,0.418597,260537.166667,0.231815,0.344878,0.205710,-17.048667,0.073662,101.531493,0.379327,0.653333,2
1,1,1922,0.938592,0.482042,165469.746479,0.237815,0.434195,0.240720,-19.275282,0.116655,100.884521,0.535549,0.140845,10
2,1,1923,0.957247,0.577341,177942.362162,0.262406,0.371733,0.227462,-14.129211,0.093949,114.010730,0.625492,5.389189,0
3,1,1924,0.940200,0.549894,191046.707627,0.344347,0.581701,0.235219,-14.231343,0.092089,120.689572,0.663725,0.661017,10
4,1,1925,0.962607,0.573863,184986.924460,0.278594,0.418297,0.237668,-14.146414,0.111918,115.521921,0.621929,2.604317,5


In [30]:
dados_anos.year.unique()

array([1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931,
       1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942,
       1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953,
       1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964,
       1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975,
       1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986,
       1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
       1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020], dtype=int64)

In [37]:
dados_anos = dados_anos[dados_anos.year >= 2000]
dados_anos = dados_anos.drop(["key", "mode"], axis=1)
dados_anos.year.unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020],
      dtype=int64)

In [38]:
dados_anos.reset_index()

,index,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,79,2000,0.289323,0.590918,242724.642638,0.625413,0.101168,0.197686,-8.247766,0.089205,118.999323,0.559475,46.684049
1,80,2001,0.286842,0.583318,240307.796010,0.626986,0.107214,0.187026,-8.305095,0.089182,117.765399,0.541479,48.750125
2,81,2002,0.282624,0.576160,239503.283000,0.641270,0.088048,0.193911,-7.686640,0.084308,119.239738,0.542397,48.655500
3,82,2003,0.256471,0.575763,244670.575230,0.660165,0.083049,0.196976,-7.485545,0.093926,120.914622,0.530504,48.626407
4,83,2004,0.280559,0.567680,237378.708037,0.648868,0.077934,0.202199,-7.601655,0.094239,121.290346,0.524489,49.273143
5,84,2005,0.255764,0.572281,237229.588205,0.653209,0.090194,0.190082,-7.466159,0.093334,121.617967,0.532531,50.953333
6,85,2006,0.279986,0.568230,234042.914359,0.650326,0.077701,0.188289,-7.265501,0.085847,121.798615,0.520028,51.313846
7,86,2007,0.254081,0.563414,241049.962564,0.668305,0.072957,0.196127,-7.044536,0.084347,124.087516,0.516794,51.075897
8,87,2008,0.249192,0.579193,240107.315601,0.671461,0.063662,0.198431,-6.843804,0.077356,123.509934,0.527542,50.630179
9,88,2009,0.261929,0.564190,238140.013265,0.670749,0.075872,0.205252,-7.046015,0.085458,123.463808,0.507170,51.440816


In [47]:
import plotly.express as px

In [41]:
fig = px.line(dados_anos, x="year", y="loudness",title="Variação do loudness conforme os anos")
fig.show()

In [42]:
import plotly.graph_objects as go

In [43]:
fig = go.Figure()

In [45]:
fig.add_trace(go.Scatter(
    x=dados_anos["year"], y=dados_anos["acousticness"], name="Acousticness"))
fig.add_trace(go.Scatter(
    x=dados_anos["year"], y=dados_anos["valence"], name="Valence"))
fig.add_trace(go.Scatter(
    x=dados_anos["year"], y=dados_anos["danceability"], name="Danceability"))
fig.add_trace(go.Scatter(
    x=dados_anos["year"], y=dados_anos["energy"], name="Energy"))
fig.add_trace(go.Scatter(
    x=dados_anos["year"], y=dados_anos["instrumentalness"], name="Instrumentalness"))
fig.add_trace(go.Scatter(
    x=dados_anos["year"], y=dados_anos["liveness"], name="Liveness"))
fig.add_trace(go.Scatter(
    x=dados_anos["year"], y=dados_anos["speechiness"], name="Speechiness"))
fig.show()

In [51]:
dados_generos.head(2)

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.3616,-31.514333,0.040567,75.336500,0.103783,27.833333
1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.1310,-16.854000,0.076817,120.285667,0.221750,52.500000


In [50]:
dados_generos["genres"].value_counts().sum()

2973

In [52]:
dados_generos1 = dados_generos.drop('genres', axis=1)
dados_generos1.head(2)

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.3616,-31.514333,0.040567,75.336500,0.103783,27.833333
1,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.1310,-16.854000,0.076817,120.285667,0.221750,52.500000


In [53]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [54]:
SEED = 1234
np.random.seed(1234)

pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=2, random_state=SEED))])
genre_embedding_pca = pca_pipeline.fit_transform(dados_generos1)
projection = pd.DataFrame(columns=['x', 'y'], data=genre_embedding_pca)

In [57]:
projection

,x,y
0,5.910268,-0.011146
1,2.787093,4.498483
2,-0.757538,-1.258495
3,1.020521,-0.931690
4,0.753911,-0.999861
...,...,...
2968,-0.475136,-0.017126
2969,-3.026756,-1.304983
2970,-0.832511,-1.089726
2971,1.774790,-3.695233


In [58]:
from sklearn.cluster import KMeans

In [59]:
k_means_pca = KMeans(n_clusters=5, verbose=True, random_state=SEED)
k_means_pca.fit(projection)
dados_generos["cluster_pca"] = k_means_pca.predict(projection)
projection["cluster_pca"] = k_means_pca.predict(projection)

Initialization complete
Iteration 0, inertia 4154.5700984627765
Iteration 1, inertia 3798.7130534776534
Iteration 2, inertia 3770.172485108021
Iteration 3, inertia 3755.705603513895
Iteration 4, inertia 3748.894007784977
Iteration 5, inertia 3740.8619665659553
Iteration 6, inertia 3727.822319268279
Iteration 7, inertia 3714.2358816774367
Iteration 8, inertia 3700.258965206951
Iteration 9, inertia 3686.150920938974
Iteration 10, inertia 3672.056798261093
Iteration 11, inertia 3650.3001487876054
Iteration 12, inertia 3628.0043713279306
Iteration 13, inertia 3600.2078071176925
Iteration 14, inertia 3578.753240143429
Iteration 15, inertia 3562.878244020139
Iteration 16, inertia 3546.506264758141
Iteration 17, inertia 3533.343538440654
Iteration 18, inertia 3522.8199747436415
Iteration 19, inertia 3517.7479107221725
Iteration 20, inertia 3516.071382630963
Iteration 21, inertia 3515.53888890532
Iteration 22, inertia 3515.1241644683128
Iteration 23, inertia 3514.8684261821413
Converged at ite

In [60]:
projection

,x,y,cluster_pca
0,5.910268,-0.011146,2
1,2.787093,4.498483,2
2,-0.757538,-1.258495,0
3,1.020521,-0.931690,3
4,0.753911,-0.999861,3
...,...,...,...
2968,-0.475136,-0.017126,1
2969,-3.026756,-1.304983,0
2970,-0.832511,-1.089726,0
2971,1.774790,-3.695233,3


In [61]:
projection["generos"] = dados_generos['genres']
projection.head()

,x,y,cluster_pca,generos
0,5.910268,-0.011146,2,21st century classical
1,2.787093,4.498483,2,432hz
2,-0.757538,-1.258495,0,8-bit
3,1.020521,-0.931690,3,[]
4,0.753911,-0.999861,3,a cappella


In [62]:
fig = px.scatter(projection, x="x", y="y", color="cluster_pca", hover_data=['x', 'y', 'generos'])
fig.show()

In [64]:
# quantidade de radio que são explicadas
pca_pipeline[1].explained_variance_ratio_.sum()

0.4927066994726641

In [66]:
# quantidade de colunas ou clusters que são explicadas
pca_pipeline[1].explained_variance_.sum()

5.421597305805697

In [67]:
dados.head(2)

,valence,year,acousticness,artists,danceability,duration_ms,energy,id,instrumentalness,liveness,loudness,name,popularity,release_date,speechiness,tempo,artists_song
12873,0.285,2000,0.00239,Coldplay,0.429,266773,0.661,3AJwUDP919kvQ9QcozQPxg,0.000121,0.2340,-7.227,Yellow,84,2000-07-10,0.0281,173.372,Coldplay - Yellow
12874,0.613,2000,0.14300,OutKast,0.843,270507,0.806,0I3q5fE6wg7LIfHGngUTnV,0.000000,0.0771,-5.946,Ms. Jackson,80,2000-10-31,0.2690,94.948,OutKast - Ms. Jackson


In [69]:
dados['artists'].value_counts()

Drake                    170
Taylor Swift             156
Eminem                   145
Kanye West               136
BTS                      122
                        ... 
Geto Boys                  1
Engelbert Humperdinck      1
Joe Walsh                  1
Misfits                    1
Alvin & The Chipmunks      1
Name: artists, Length: 1563, dtype: int64

In [70]:
dados['artists_song'].value_counts()

Coldplay - Yellow                                       1
Iron Maiden - Ghost of the Navigator - 2015 Remaster    1
Marco Barrientos - Al Que Esta Sentado en el Trono      1
Rage Against The Machine - Kick out the Jams            1
The Cure - Out of This World                            1
                                                       ..
Set It Off - The Haunting                               1
J. Cole - St. Tropez                                    1
Justin Bieber - All That Matters                        1
Chris Brown - Drunk Texting (feat. Jhené Aiko)          1
Eminem - Darkness                                       1
Name: artists_song, Length: 21546, dtype: int64

In [71]:
from sklearn.preprocessing import OneHotEncoder

In [72]:
ohe = OneHotEncoder(dtype=int)

In [93]:
colunas_ohe = ohe.fit_transform(dados[['artists']]).toarray()
dados2 = dados.drop('artists', axis=1)
dados_music_dummies = pd.concat([dados2, pd.DataFrame(colunas_ohe, columns=ohe.get_feature_names_out(['artists']))], axis=1)

KeyboardInterrupt: 

In [ ]:
pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=0.7, random_state=SEED))])
music_embedding_pca = pca_pipeline.fit_transform(dados_music_dummies.drop(['id', 'name', 'artists_song', 'release_date'], axis=1))
projection = pd.DataFrame(data=music_embedding_pca)

ValueError: could not convert string to float: '2000-07-10'